In [2]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
import pickle
import sys
from tensorflow.keras.models import load_model
try:
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
except Exception as e:
    print(e)
    sys.exit(1)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Paths
# Get the directory of the current script
script_dir = os.path.dirname(os.path.abspath(__file__))

# Paths with the script directory as the base
data_path = os.path.join(
    script_dir, "..", "RML2016.10a_dict.pkl"
)  # One level up from the script's directory

data_path = os.path.join(script_dir, "..", "RML2016.10a_dict.pkl")  
model_path = os.path.join(script_dir, "..", "RNN", "models", "rnn_lstm_w_SNR_5_2_1.keras")
stats_path = os.path.join(script_dir, "..", "RNN", "stats", "confusion_matrix", "rnn_lstm_w_SNR_5_2_1.png")

# Load Data
with open(data_path, 'rb') as f:
    data = pickle.load(f, encoding='latin1')

# Prepare Data
X, y = [], []
for (mod_type, snr), signals in data.items():
    for signal in signals:
        iq_signal = np.vstack([signal[0], signal[1]]).T
        snr_signal = np.full((128, 1), snr)
        combined_signal = np.hstack([iq_signal, snr_signal])
        X.append(combined_signal)
        y.append(mod_type)

X = np.array(X)
y = np.array(y)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)
X_test = X_test.reshape(-1, X_test.shape[1], X_test.shape[2])

# Load Model
if os.path.exists(model_path):
    print(f"Loading model from {model_path}")
    model = load_model(model_path)
else:
    raise FileNotFoundError(f"Model file not found at {model_path}")


# Generate Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=label_encoder.classes_)
disp.plot(cmap=plt.cm.Blues)

# # Save the confusion matrix as a PNG file
# plt.title("Confusion Matrix of Model Predictions")
# plt.savefig(stats_path, format="png")
# print(f"Confusion matrix saved to {stats_path}")

plt.show()



/usr/local/lib/python3.8/dist-packages/sklearn/__check_build/../../scikit_learn.libs/libgomp-d22c30c5.so.1.0.0: cannot allocate memory in static TLS block
___________________________________________________________________________
Contents of /usr/local/lib/python3.8/dist-packages/sklearn/__check_build:
__init__.py               _check_build.cpython-38-aarch64-linux-gnu.so__pycache__
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to build the package before using it: run `python setup.py install` or
`make` in the source directory.

If you have used an installer, please check that it is suited for your
Python version, your operating system and your platform.


AttributeError: 'tuple' object has no attribute 'tb_frame'